In [ ]:
! pip install nbformat

In [2]:
import nbformat
import re
from copy import deepcopy

In [3]:

def renumber_markdown_headings(nb_path, output_path=None):
    with open(nb_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)

    numbered_nb = deepcopy(nb)
    counters = [0] * 6  # For # to ######

    heading_pattern = re.compile(r'^(#{1,6})\s+(.*)')

    for cell in numbered_nb['cells']:
        if cell['cell_type'] != 'markdown':
            continue

        new_lines = []
        for line in cell['source'].split('\n'):
            match = heading_pattern.match(line)
            if match:
                level = len(match.group(1))
                title = match.group(2).strip()

                # Update heading counters
                counters[level - 1] += 1
                for i in range(level, 6):
                    counters[i] = 0

                numbering = '.'.join(str(counters[i]) for i in range(level) if counters[i] > 0)
                new_line = f"{'#' * level} {numbering} {title}"
                new_lines.append(new_line)
            else:
                new_lines.append(line)

        cell['source'] = '\n'.join(new_lines)

    # Save to output path or overwrite original
    save_path = output_path or nb_path
    with open(save_path, 'w', encoding='utf-8') as f:
        nbformat.write(numbered_nb, f)

    print(f"Notebook saved with numbered headings at: {save_path}")


In [4]:
renumber_markdown_headings('../Notebooks/25/VQA_RAG/SpiQA_dataset.ipynb', output_path='../Notebooks/25/VQA_RAG/SpiQA_dataset_n.ipynb')

Notebook saved with numbered headings at: ../Notebooks/25/VQA_RAG/SpiQA_dataset_n.ipynb
